In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta
import data_preperation as prep
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

#example of how to call preparation.py
import data_process as proc
import data_preperation as prep
from utils_scenario import *


In [ ]:
#example of how to use data_process.py

#call data_preperation.py to get data 
measurement=prep.prepare_measurement()
forecast = prep.prepare_forecast()
#keep useful columns 
measurement= measurement[['speed', 'cos_wind_dir', 'sin_wind_dir', 'temp', 'radiation', 'precip', 'season', 'am']]

#call data_process.py
steps_in = 48
steps_out = 12
x_df, y_df, x, y_speed = proc.prepare_x_y(measurement, forecast, steps_in, steps_out, 'speed')
x_df, y_df, x, y_cos = proc.prepare_x_y(measurement, forecast, steps_in, steps_out, 'cos_wind_dir')
x_df, y_df, x, y_sin = proc.prepare_x_y(measurement, forecast, steps_in, steps_out, 'sin_wind_dir')
y_scenarios = get_all_scenarios(y_speed, y_cos, y_sin, b_scenarios = True)
y_dangerous = get_all_dangerous_scenarios(y_speed, y_cos, y_sin)
X_train, X_test, y_train_dangerous, y_test_dangerous = train_test_split(x, y_dangerous, test_size=0.2, shuffle = False)
_, _, y_train_scenarios, y_test_scenarios = train_test_split(x, y_scenarios, test_size=0.2, shuffle = False)
_, _, y_train_speed, y_test_speed = train_test_split(x, y_speed, test_size=0.2, shuffle = False)
_, _, y_train_cos, y_test_cos = train_test_split(x, y_cos, test_size=0.2, shuffle = False)
_, _, y_train_sin, y_test_sin = train_test_split(x, y_sin, test_size=0.2, shuffle = False)

read csv semester csv files from 2015s2 to 2020s1
smooth wind direction
generate seasonality categorical feature
generate am/pm categorical feature
reading forecast data
smooth wind direction


In [ ]:
#Regressors
xg_speed = XGBRegressor(max_depth = 5)
xg_speed.fit(X_train, y_train_speed)
y_hat_speed = xg_speed.predict(X_test)

xg_cos = XGBRegressor(max_depth = 5)
xg_cos.fit(X_train, y_train_cos)
y_hat_cos = xg_cos.predict(X_test)

xg_sin = XGBRegressor(max_depth = 5)
xg_sin.fit(X_train, y_train_sin)
y_hat_sin = xg_sin.predict(X_test)

y_hat_scenario = get_all_scenarios(y_hat_speed, y_hat_cos, y_hat_sin, b_scenarios=True)
y_hat_dangerous = get_all_dangerous_scenarios(y_hat_speed, y_hat_cos, y_hat_sin)

In [ ]:
y_hat_scenario_from_regression = get_all_scenarios(y_hat_speed, y_hat_cos, y_hat_sin, b_scenarios=True)
y_hat_dangerous_from_regression = get_all_dangerous_scenarios(y_hat_speed, y_hat_cos, y_hat_sin)

In [ ]:
#Classifiers
xg_dangerous_scenario = XGBClassifier(max_depth = 5)
xg_dangerous_scenario.fit(X_train, y_train_dangerous)
y_hat_dangerous = xg_dangerous_scenario.predict(X_test)

xg_scenario = XGBClassifier(max_depth = 5)
xg_scenario.fit(X_train, y_train_scenarios)
y_hat_scenarios = xg_scenario.predict(X_test)

In [ ]:
#Baselines
y_test_baseline_speed, y_test_baseline_cos_wind, y_test_baseline_sin_wind, y_baseline_dangerous_scenarios, y_baseline_scenarios = get_baselines(x_df, x)

In [ ]:
#Scores
#print("MSE speed is: ", mean_squared_error(y_test_speed, y_hat_speed))
print("MAE speed is: ", mean_absolute_error(y_test_speed, y_hat_speed))
#print("MSE baseline speed is: ", mean_squared_error(y_test_speed, baseline_speed))
print("MAE baseline speed is: ", mean_absolute_error(y_test_speed, y_test_baseline_speed))

#print("MSE cos is: ", mean_squared_error(y_te_cos, y_hat_cos))
print("MAE cos is: ", mean_absolute_error(y_test_cos, y_hat_cos))
#print("MSE baseline cos is: ", mean_squared_error(y_te_cos, baseline_cos))
print("MAE baseline cos is: ", mean_absolute_error(y_test_cos, y_test_baseline_cos_wind))

#print("MSE sin is: ", mean_squared_error(y_te_sin, y_hat_sin))
print("MAE sin is: ", mean_absolute_error(y_test_sin, y_hat_sin))
#print("MSE baseline sin is: ", mean_squared_error(y_te_sin, baseline_sin))
print("MAE baseline sin is: ", mean_absolute_error(y_test_sin, y_test_baseline_sin_wind))

In [ ]:
print("Accuracy for classification from regression is:", accuracy_score(y_test_scenarios, y_hat_scenario_from_regression))
print("Accuracy for baseline is:", accuracy_score(y_test_scenarios, y_baseline_scenarios))
print("Accuracy for classification based prediction is:", accuracy_score(y_test_scenarios, y_hat_scenarios))

In [131]:
y_hat_dangerous.shape

(8193,)

# Metrics

In [39]:
print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

           1       0.87      0.97      0.92      6671
           2       0.49      0.34      0.40      1015
           3       0.71      0.02      0.03       308
           4       0.21      0.06      0.09       126
           5       0.00      0.00      0.00        45
           6       0.07      0.03      0.04        32

   micro avg       0.83      0.83      0.83      8197
   macro avg       0.39      0.23      0.25      8197
weighted avg       0.80      0.83      0.80      8197



In [113]:
print(classification_report(y_test_dangerous, y_hat_dangerous))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      7847
           1       0.35      0.12      0.18       195

   micro avg       0.97      0.97      0.97      8042
   macro avg       0.66      0.56      0.58      8042
weighted avg       0.96      0.97      0.97      8042



In [133]:
print(classification_report(y_test_dangerous, y_hat_dangerous))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      7990
           1       0.38      0.12      0.19       203

   micro avg       0.97      0.97      0.97      8193
   macro avg       0.68      0.56      0.59      8193
weighted avg       0.96      0.97      0.97      8193



In [58]:
print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

           1       0.87      0.96      0.92      6667
           2       0.48      0.36      0.41      1015
           3       0.46      0.04      0.08       308
           4       0.16      0.03      0.05       126
           5       0.07      0.02      0.03        45
           6       0.21      0.09      0.13        32

   micro avg       0.83      0.83      0.83      8193
   macro avg       0.38      0.25      0.27      8193
weighted avg       0.79      0.83      0.80      8193



In [111]:
print("Accuracy for classification based prediction is:", accuracy_score(y_test_dangerous, y_hat_dangerous))

Accuracy for classification based prediction is: 0.9731410096990798


In [132]:
print("Accuracy for classification based prediction is:", accuracy_score(y_test_dangerous, y_hat_dangerous))

Accuracy for classification based prediction is: 0.973391919931649


In [124]:
X_test.shape

(8193, 400)

In [ ]:
print(classification_report(y_test, y_hat))

In [153]:
def get_baselines(x_df, x, test_size = 0.2):
    y_baseline_speed = np.array(x_df['speed_forecast'])
    y_baseline_cos_wind = np.array(x_df['cos_wind_dir_forecast'])
    y_baseline_sin_wind = np.array(x_df['sin_wind_dir_forecast'])
    _, _, _, y_test_baseline_speed = train_test_split(x, y_baseline_speed, test_size = test_size, shuffle = False)
    _, _, _, y_test_baseline_cos_wind = train_test_split(x, y_baseline_cos_wind, test_size = test_size, shuffle = False)
    _, _, _, y_test_baseline_sin_wind = train_test_split(x, y_baseline_sin_wind, test_size = test_size, shuffle = False)
    y_baseline_dangerous_scenarios = get_all_dangerous_scenarios(y_test_baseline_speed, y_test_baseline_cos_wind, y_baseline_sin_wind)
    y_baseline_scenarios = get_all_scenarios(y_test_baseline_speed, y_test_baseline_cos_wind, y_baseline_sin_wind, b_scenarios=True)
    return y_test_baseline_speed, y_test_baseline_cos_wind, y_test_baseline_sin_wind, y_baseline_dangerous_scenarios, y_baseline_scenarios

In [154]:
y_test_baseline_speed, y_test_baseline_cos_wind, y_test_baseline_sin_wind, y_baseline_dangerous_scenarios, y_baseline_scenarios = get_baselines(x_df, x)

In [136]:
print("Accuracy for classification based prediction is:", accuracy_score(y_test_dangerous, y_baseline_dangerous_scenarios))

Accuracy for classification based prediction is: 0.9443427316001465
